In [ ]:
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import cv2
import os
from tqdm import tqdm_notebook as tqdm
import numpy as np


In [ ]:
drive.mount("/content/gdrive")
path = "/content/gdrive/MyDrive/aptos2019-blindness-detection/"
df = pd.read_csv(path+"train.csv", sep=",")
train_images = os.listdir(path + "train_images")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
number_of_train_image = 200
steps_per_epoch = int(number_of_train_image * 0.20)

In [ ]:
img_list = []

In [ ]:
for i in tqdm(train_images[:number_of_train_image]):
  img = cv2.imread(path + "train_images/"+ i)
  img = cv2.resize(img, (80,80))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  copy = img.copy()
  copy = cv2.cvtColor(copy, cv2.COLOR_RGB2GRAY)
  blurred = cv2.GaussianBlur(copy,(5,5), 0)
  thresh = cv2.threshold(blurred, 18, 255, cv2.THRESH_BINARY)[1]
  contour = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  contour = contour[0][0]
  contour = contour[:,0,:]
  x1 = tuple(contour[contour[:,0].argmin()])[0]
  y1 = tuple(contour[contour[:,1].argmin()])[1]
  x2 = tuple(contour[contour[:,0].argmax()])[0]
  y2 = tuple(contour[contour[:,1].argmax()])[1]
  x = int(x2-x1)*4//100
  y = int(y2-y1)*5//100
  copy_2 = img.copy()
  if x2-x1 > 100 and y2-y1 > 100:
    copy_2 = copy_2[y1+y:y2-y, x1+x:x2-x]
    copy_2 = cv2.resize(copy_2, (80,80))
  lab = cv2.cvtColor(copy_2, cv2.COLOR_RGB2LAB)
  l, a, b = cv2.split(lab)
  clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=((8,8)))
  cl = clahe.apply(l)
  l_img = cv2.merge((cl, a, b))
  last = cv2.cvtColor(l_img, cv2.COLOR_LAB2RGB)
  med_last = cv2.medianBlur(last, 3)
  background = cv2.medianBlur(last, 37)
  mask = cv2.addWeighted(med_last,1, background,-1,255)
  final_img = cv2.bitwise_and(mask, med_last)
  img_list.append(final_img)
# fig = plt.figure(figsize=(20,12))
# for i in range (12):
#   img = img_list[i]
#   fig.add_subplot(3, 4, i+1)
#   plt.imshow(img)

# plt.tight_layout()

<ipython-input-7-11af53fd5d74>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(train_images[:number_of_train_image]):


  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
y_train = pd.get_dummies(df["diagnosis"]).values
y_train_last = np.ones(y_train.shape, dtype="uint8")
y_train_last[:,4] = y_train[:,4]
for i in range(3, -1, -1):
  y_train_last[:,i] = np.logical_or(y_train[:,i], y_train_last[:,i+1])

In [ ]:
x_train = np.array(img_list)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    x_train,
    y_train_last[:number_of_train_image],
    test_size = 0.15,
    random_state = 2019,
    shuffle = True
)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)

In [ ]:
data_generator = datagen.flow(x_train, y_train, batch_size=2, seed=2020)

In [ ]:
# transfer learning
# EfficientNet - B5 Model
!pip install efficientnet
!pip install tensorflow==2.6.0
!pip install keras~=2.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 5.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 458.4 MB 27 kB/s 
     |████████████████████████████████| 132 kB 47.0 MB/s 
     |████████████████████████████████| 14.9 MB 28.7 MB/s 
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30694 sha256=e0bbdac0f1c8aaebb7804f24118d9bac0624067fd1f70f07a2208eea3162c753
  Stored in directory: /root/.cache/pip/wheels/f1/60/77/22b9b5887bd47801796a856f47650d9789c74dc3161a26d608
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4849 sha256=fa9bd8b1a2f2a35e8dd6cfc9378786731e5843827b7bef061ec4fc97f5b8fcae
  Stored in directory: /root/.cache/pip/wheels/a0/16/9c/5473df82468f958445479c59e784896fa24f4a5fc024b0f501
  Created wheel for wrapt: filename=wrapt-1.12.1-cp38-cp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 16.3 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0


In [ ]:
import tensorflow as tf
# from tensorflow.keras.applications.efficientnet import EfficientNetB5

In [ ]:
model = tf.keras.applications.EfficientNetB5()
model_2 = tf.keras.applications.EfficientNetB5(weights='imagenet',include_top=False, input_shape=(80,80,3))

115277824/115263384 [==============================] - 4s 0us/step


In [ ]:
# model = tf.keras.applications.efficientnet.EfficientNetB5()
# model_2 = tf.keras.applications.efficientnet.EfficientNetB5(weights='imagenet',include_top=False, input_shape=(400,400,3))

In [ ]:
from tensorflow import keras, saved_model
from tensorflow.keras import layers

In [ ]:
model_keras = keras.Sequential()

In [ ]:
model_keras.add(model_2)

In [ ]:
model_keras.add(layers.GlobalAveragePooling2D())
model_keras.add(layers.Dropout(0.5))
model_keras.add(layers.Dense(5, activation='sigmoid'))

In [ ]:
opti = tf.optimizers.Adam(learning_rate=0.00005)

In [ ]:
model_keras.compile(loss='binary_crossentropy', optimizer=opti, metrics=['accuracy'])

In [ ]:
lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                          patience=3,
                                          verbose=1,
                                          mode='auto',
                                          factor=0.25,
                                          min_lr=0.000001)

In [ ]:
model_keras.fit(data_generator,
                steps_per_epoch = steps_per_epoch,
                epochs=15,
                validation_data = (x_val, y_val),
                callbacks=[lr])

Epoch 1/20
40/40 [==============================] - 26s 654ms/step - loss: 0.3975 - accuracy: 0.9875 - val_loss: 0.6583 - val_accuracy: 0.6667
Epoch 2/20
40/40 [==============================] - 26s 652ms/step - loss: 0.4039 - accuracy: 0.9875 - val_loss: 0.6640 - val_accuracy: 0.5667
Epoch 3/20
40/40 [==============================] - 26s 640ms/step - loss: 0.4057 - accuracy: 1.0000 - val_loss: 0.6552 - val_accuracy: 0.6667
Epoch 4/20
40/40 [==============================] - 25s 637ms/step - loss: 0.4006 - accuracy: 1.0000 - val_loss: 0.6540 - val_accuracy: 0.6333
Epoch 5/20
40/40 [==============================] - 30s 751ms/step - loss: 0.3687 - accuracy: 1.0000 - val_loss: 0.6396 - val_accuracy: 0.7333
Epoch 6/20
40/40 [==============================] - 27s 666ms/step - loss: 0.3990 - accuracy: 1.0000 - val_loss: 0.6329 - val_accuracy: 0.8667
Epoch 7/20
40/40 [==============================] - 25s 633ms/step - loss: 0.4184 - accuracy: 1.0000 - val_loss: 0.6244 - val_accuracy: 0.8000

In [ ]:
score = model_keras.evaluate(x_val, y_val, verbose=0)
print('Test accuarcy: {:0.2f}%'.format(score[1] * 100))

Test accuarcy: 86.67%


In [ ]:
import os

In [ ]:
MODEL_DIR='tf_model'
version = "1"
export_path = os.path.join(MODEL_DIR, str(version))

In [ ]:
model_keras.save(export_path, save_format="tf")
print('\nexport_path = {}'.format(export_path))
!dir {export_path}

/usr/local/lib/python3.8/dist-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '



export_path = tf_model/1
assets	keras_metadata.pb  saved_model.pb  variables


In [ ]:
model_keras.save_weights("weights.h5")

In [ ]:
model_json=model_keras.to_json()

In [ ]:
with open ("/content/model.json", "w") as json_file:
  json_file.write(model_json)

In [ ]:
!zip -r model.zip model/

	zip warning: name not matched: model/

zip error: Nothing to do! (try: zip -r model.zip . -i model/)


In [ ]:
def converter_2(i):
  img = cv2.imread(i)
  img = cv2.resize(img, (80,80))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  copy = img.copy()
  copy = cv2.cvtColor(copy, cv2.COLOR_RGB2GRAY)
  blurred = cv2.GaussianBlur(copy,(5,5), 0)
  thresh = cv2.threshold(blurred, 18, 255, cv2.THRESH_BINARY)[1]
  contour = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  contour = contour[0][0]
  contour = contour[:,0,:]
  x1 = tuple(contour[contour[:,0].argmin()])[0]
  y1 = tuple(contour[contour[:,1].argmin()])[1]
  x2 = tuple(contour[contour[:,0].argmax()])[0]
  y2 = tuple(contour[contour[:,1].argmax()])[1]
  x = int(x2-x1)*4//100
  y = int(y2-y1)*5//100
  copy_2 = img.copy()
  if x2-x1 > 100 and y2-y1 > 100:
    copy_2 = copy_2[y1+y:y2-y, x1+x:x2-x]
    copy_2 = cv2.resize(copy_2, (80,80))
  lab = cv2.cvtColor(copy_2, cv2.COLOR_RGB2LAB)
  l, a, b = cv2.split(lab)
  clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=((8,8)))
  cl = clahe.apply(l)
  l_img = cv2.merge((cl, a, b))
  last = cv2.cvtColor(l_img, cv2.COLOR_LAB2RGB)
  med_last = cv2.medianBlur(last, 3)
  background = cv2.medianBlur(last, 37)
  mask = cv2.addWeighted(med_last,1, background,-1,255)
  final_img = cv2.bitwise_and(mask, med_last)
  
  return final_img

In [ ]:
a = converter_2("/content/fundus.jpeg")

In [ ]:
a.shape

(80, 80, 3)

In [ ]:
a = np.expand_dims(a, axis=0)

In [ ]:
a

array([[[[11,  7, 10],
         [11,  7, 10],
         [11,  6, 10],
         ...,
         [11,  6, 10],
         [11,  6, 10],
         [11,  6, 10]],

        [[11,  6, 10],
         [11,  6, 10],
         [11,  6, 10],
         ...,
         [11,  6, 10],
         [11,  6, 10],
         [11,  6, 10]],

        [[11,  6, 10],
         [11,  6, 10],
         [11,  6, 10],
         ...,
         [11,  6, 10],
         [11,  6, 10],
         [11,  6, 10]],

        ...,

        [[11,  6, 10],
         [11,  6, 10],
         [11,  6, 10],
         ...,
         [11,  6, 10],
         [11,  6, 10],
         [11,  6, 10]],

        [[11,  6, 10],
         [11,  6, 10],
         [11,  6, 10],
         ...,
         [11,  6, 10],
         [11,  6, 10],
         [11,  6, 10]],

        [[11,  6, 10],
         [11,  6, 10],
         [11,  6, 10],
         ...,
         [11,  6, 10],
         [11,  6, 10],
         [11,  6, 10]]]], dtype=uint8)

In [ ]:
predicted = model_keras.predict(a)
predicted

array([[0.47449797, 0.6171645 , 0.58186203, 0.49321425, 0.46151182]],
      dtype=float32)

In [ ]:
predicted_class = np.argmax(predicted)
predicted_class

1

In [ ]:
a_list = a.tolist()

In [ ]:
json_dict = {"instances": a_list}

In [ ]:
import json
with open("data_file.json", "w") as write_file:
    json.dump(json_dict, write_file)